# Setup

On some systems, this cell needs to be run twice to get the correct settings of matplotlib.

In [ ]:
from rrk import *
from scipy import interpolate
import time
import scipy as sp

plt.rc("font", family="serif", size=16.)

# Lotka Volterra System: Used in the Paper

In [ ]:
a = 1.0
b = 1.0
c = 1.0
d = 1.0

def f(w):
    return np.array([w[0]*(a-b*w[1]), w[1]*(c*w[0]-d)])

def eta(w):
    return c*w[0] - d*np.log(w[0]) + b*w[1] - a*np.log(w[1])

def d_eta(w):
    return np.array([c - d/w[0], b - a/w[1]])

def compute_and_plot(dt, t_final, erk):
    u0 = [1., 2.]
    
    time_tmp = time.time()
    tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0, t_final,
                                relaxed=False, method="brentq", newdt=True)
    time_tmp = time.time() - time_tmp
    print("Baseline scheme:   %.2e s" % time_tmp)

    plt.close("all");
    fig_phase = plt.figure(figsize=(4,4))
    plt.plot(uu[0,:], uu[1,:], 'o', label="Baseline", markersize=2., lw=0.5)
    plt.xlabel("$u_1$"); plt.ylabel("$u_2$")
    plt.axis('equal')

    time_tmp = time.time()
    tt_r, uu_r = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0, t_final,
                                relaxed=True, method="brentq", newdt=True)
    time_tmp = time.time() - time_tmp
    print("Relaxation scheme: %.2e s" % time_tmp)

    plt.plot(uu_r[0,:], uu_r[1,:], 'v', label="Relaxation", markersize=2)

    fig_eta = plt.figure()
    H = [eta(uu[:,i]) for i in range(uu.shape[1])]
    plt.plot(tt, H - H[0], label="Baseline")
    H_r = [eta(uu_r[:,i]) for i in range(uu_r.shape[1])]
    plt.plot(tt_r, H_r - H_r[0], label="Relaxation")
    plt.xlabel("$t$"); plt.ylabel("$H(u(t)) - H(u(0))$"); plt.xlim(tt[0], tt[-1])
    plt.xscale("symlog")
    plt.yscale("symlog", linthreshy=1.e-14)
    
    ax = plt.gca()
    fig_legend = plt.figure()
    handles, labels = ax.get_legend_handles_labels()
    plt.figlegend(handles, labels, loc="center", ncol=6)
    
    return tt, uu, tt_r, uu_r, fig_phase, fig_eta, fig_legend


erk = rk4; dt = 0.85; t_final = 500

tt, uu, tt_r, uu_r, fig_phase, fig_eta, fig_legend = compute_and_plot(dt, t_final, erk)

In [ ]:
fig_phase.savefig("../figures/LotkaVolterra__RK4_phase_space.pdf", bbox_inches="tight")
fig_eta.savefig("../figures/LotkaVolterra__RK4_Hamiltonian.pdf", bbox_inches="tight")
fig_legend.savefig("../figures/LotkaVolterra__legend.pdf", bbox_inches="tight")

# Hénon-Heiles System

In [ ]:
def f(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    
    dq1 = p1
    dq2 = p2
    dp1 = -(q1 + 2*q1*q2)
    dp2 = -(q2 + q1*q1 - q2*q2)
    return np.array([dq1, dq2, dp1, dp2])

def eta(w):
    q1 = w[0]
    q2 = w[1]
    p1 = w[2]
    p2 = w[3]
    return 0.5*(p1*p1 + p2*p2) + 0.5*(q1*q1 + q2*q2) + q1*q1*q2 - q2*q2*q2/3

def d_eta(w):
    fw = f(w)
    dq1 = fw[0]
    dq2 = fw[1]
    dp1 = fw[2]
    dp2 = fw[3]
    return np.array([-dp1, -dp2, dq1, dq2])


def compute_and_plot(u0, dt, t_final, erk):
    time_tmp = time.time()
    tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0, t_final,
                                relaxed=False, method="brentq", newdt=True)
    time_tmp = time.time() - time_tmp
    print("Baseline scheme:   %.2e s" % time_tmp)

    plt.close("all");

    q1 = interpolate.InterpolatedUnivariateSpline(tt, uu[0,:])
    q2 = interpolate.InterpolatedUnivariateSpline(tt, uu[1,:])
    p1 = interpolate.InterpolatedUnivariateSpline(tt, uu[2,:])
    p2 = interpolate.InterpolatedUnivariateSpline(tt, uu[3,:])

    fig_poincare = plt.figure(figsize=(4,4))
    t = q1.roots()
    plt.plot(q2(t), p2(t), label="Baseline", linestyle="None", marker=".", markersize=3)
    plt.xlabel("$q_2$"); plt.ylabel("$p_2$")
    plt.axis('equal')

    time_tmp = time.time()
    tt_r, uu_r = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0, t_final,
                                relaxed=True, method="brentq", newdt=True)
    time_tmp = time.time() - time_tmp
    print("Relaxation scheme: %.2e s" % time_tmp)

    q1_r = interpolate.InterpolatedUnivariateSpline(tt_r, uu_r[0,:])
    q2_r = interpolate.InterpolatedUnivariateSpline(tt_r, uu_r[1,:])
    p1_r = interpolate.InterpolatedUnivariateSpline(tt_r, uu_r[2,:])
    p2_r = interpolate.InterpolatedUnivariateSpline(tt_r, uu_r[3,:])

    t_r = q1_r.roots()
    plt.plot(q2_r(t_r), p2_r(t_r), label="Relaxation", linestyle="None", marker=".", markersize=3)

    fig_eta = plt.figure()
    H = [eta(uu[:,i]) for i in range(uu.shape[1])]
    plt.plot(tt, H - H[0], label="Baseline")
    H_r = [eta(uu_r[:,i]) for i in range(uu_r.shape[1])]
    plt.plot(tt_r, H_r - H_r[0], label="Relaxation")
    plt.xlabel("$t$"); plt.ylabel("$H(u(t)) - H(u(0))$"); plt.xlim(tt[0], tt[-1])
    plt.xscale("symlog")
    plt.yscale("symlog", linthreshy=1.e-14)
    
    ax = plt.gca()
    fig_legend = plt.figure()
    handles, labels = ax.get_legend_handles_labels()
    plt.figlegend(handles, labels, loc="center", ncol=6)
    
    return tt, uu, tt_r, uu_r, fig_poincare, fig_eta, fig_legend


In [ ]:
u0 = np.array([0.12, 0.12, 0.12, 0.12])
erk = ssp33; dt = 0.1; t_final = 10000

tt, uu, tt_r, uu_r, fig_poincare, fig_eta, fig_legend = compute_and_plot(u0, dt, t_final, erk)

In [ ]:
fig_poincare.savefig("../figures/HenonHeiles__ssp33_Poincare.pdf", bbox_inches="tight")
fig_eta.savefig("../figures/HenonHeiles__ssp33_Hamiltonian.pdf", bbox_inches="tight")
fig_legend.savefig("../figures/HenonHeiles__legend.pdf", bbox_inches="tight")

In [ ]:
u0 = np.array([0.0, 0.0, np.sqrt(2.0*0.15925), 0.0])
erk = ssp33; dt = 0.1; t_final = 30000

tt, uu, tt_r, uu_r, fig_poincare, fig_eta = compute_and_plot(u0, dt, t_final, erk)

In [ ]:
x, y = fig_poincare.axes[0].lines[0].get_data()
filename = "../figures/HenonHeiles_chaotic__ssp33_baseline"
np.savetxt(filename + ".txt", np.vstack((x, y)))

x, y = fig_poincare.axes[0].lines[1].get_data()
filename = "../figures/HenonHeiles_chaotic__ssp33_relaxation"
np.savetxt(filename + ".txt", np.vstack((x, y)))

In [ ]:
import subprocess

plt.figure(figsize=(4,4))
x, y = fig_poincare.axes[0].lines[0].get_data()
plt.plot(x, y, linestyle="None", marker=".", markersize=0.001, markeredgewidth=1.0, color="black")
plt.xlabel("$q_2$"); plt.ylabel("$p_2$")
plt.xlim(-0.6, 1.0)
plt.ylim(-0.8, 0.8)
filename = "../figures/HenonHeiles_chaotic__ssp33_baseline"
plt.savefig(filename + "_tmp.pdf", bbox_inches="tight")
subprocess.run(["pdf2ps", filename + "_tmp.pdf", filename + "_tmp.ps"])
subprocess.run(["ps2pdf", filename + "_tmp.ps", filename + ".pdf"])
subprocess.run(["rm", filename + "_tmp.ps", filename + "_tmp.pdf"])

plt.figure(figsize=(4,4))
x, y = fig_poincare.axes[0].lines[0].get_data()
plt.plot(x, y, linestyle="None", marker=".", markersize=0.001, markeredgewidth=0.5, color="black")
plt.xlabel("$q_2$"); plt.ylabel("$p_2$")
plt.xlim(-0.6, 1.0)
plt.ylim(-0.8, 0.8)
filename = "../figures/HenonHeiles_chaotic__ssp33_baseline"
plt.savefig(filename + "_tmp.pdf", bbox_inches="tight")
subprocess.run(["pdf2ps", filename + "_tmp.pdf", filename + "_tmp.ps"])
subprocess.run(["ps2pdf", filename + "_tmp.ps", filename + "__small.pdf"])
subprocess.run(["rm", filename + "_tmp.ps", filename + "_tmp.pdf"])

plt.figure(figsize=(4,4))
x, y = fig_poincare.axes[0].lines[1].get_data()
plt.plot(x, y, linestyle="None", marker=".", markersize=0.001, markeredgewidth=1.0, color="black")
plt.xlabel("$q_2$"); plt.ylabel("$p_2$")
plt.xlim(-0.6, 1.0)
plt.ylim(-0.8, 0.8)
filename = "../figures/HenonHeiles_chaotic__ssp33_relaxation"
plt.savefig(filename + "_tmp.pdf", bbox_inches="tight")
subprocess.run(["pdf2ps", filename + "_tmp.pdf", filename + "_tmp.ps"])
subprocess.run(["ps2pdf", filename + "_tmp.ps", filename + ".pdf"])
subprocess.run(["rm", filename + "_tmp.ps", filename + "_tmp.pdf"])

plt.figure(figsize=(4,4))
x, y = fig_poincare.axes[0].lines[1].get_data()
plt.plot(x, y, linestyle="None", marker=".", markersize=0.001, markeredgewidth=1.0, color="black")
plt.xlabel("$q_2$"); plt.ylabel("$p_2$")
plt.xlim(-0.6, 1.0)
plt.ylim(-0.8, 0.8)
filename = "../figures/HenonHeiles_chaotic__ssp33_relaxation"
plt.savefig(filename + "_tmp.pdf", bbox_inches="tight")
subprocess.run(["pdf2ps", filename + "_tmp.pdf", filename + "_tmp.ps"])
subprocess.run(["ps2pdf", filename + "_tmp.ps", filename + "__small.pdf"])
subprocess.run(["rm", filename + "_tmp.ps", filename + "_tmp.pdf"])

# Undamped Duffing Oscillator with Initial Values near the Separatrix

In [ ]:
def f(w):
    q = w[0]
    p = w[1]
    return np.array([p, q - q*q*q])

def eta(w):
    q = w[0]
    p = w[1]
    return 0.5*p*p - 0.5*q*q + 0.25*q*q*q*q

def d_eta(w):
    fw = f(w)
    dq = fw[0]
    dp = fw[1]
    return np.array([-dp, dq])


def compute_and_plot(dt, t_final, erk):
    u0 = np.array([1.4142, 0.0])
    
    time_tmp = time.time()
    tt, uu = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0, t_final,
                                relaxed=False, method="brentq", newdt=True)
    time_tmp = time.time() - time_tmp
    print("Baseline scheme:   %.2e s" % time_tmp)

    plt.close("all");
    fig_phase = plt.figure(figsize=(6,6))
    plt.plot(uu[0,:], uu[1,:], 'o', label="Baseline", markersize=2, lw=0.5)
    plt.xlabel("$q$"); plt.ylabel("$p$")
    plt.axis('equal')

    time_tmp = time.time()
    tt_r, uu_r = convex_relaxed_ERK(erk, dt, f, eta, d_eta, u0, t_final,
                                relaxed=True, method="brentq", newdt=True)
    
    time_tmp = time.time() - time_tmp
    print("Relaxation scheme: %.2e s" % time_tmp)

    plt.plot(uu_r[0,:], uu_r[1,:], 'v', label="Relaxation", markersize=2., lw=0.5)

    fig_eta = plt.figure()
    H = [eta(uu[:,i]) for i in range(uu.shape[1])]
    plt.plot(tt, H - H[0], label="Baseline")
    H_r = [eta(uu_r[:,i]) for i in range(uu_r.shape[1])]
    plt.plot(tt_r, H_r - H_r[0], label="Relaxation")
    plt.xscale("symlog")
    plt.yscale("symlog", linthreshy=1.e-14)
    
    ax = fig_phase.axes[0]
    fig_legend = plt.figure()
    handles, labels = ax.get_legend_handles_labels()
    plt.figlegend(handles, labels, loc="center", ncol=6)
    
    return tt, uu, tt_r, uu_r, fig_phase, fig_eta, fig_legend
    

erk = rk4; dt = 0.5; t_final = 500

tt, uu, tt_r, uu_r, fig_phase, fig_eta, fig_legend = compute_and_plot(dt, t_final, erk)

In [ ]:
erk = rk4; dt = 0.5; t_final = 500
tt, uu, tt_r, uu_r, fig_phase, fig_eta, fig_legend = compute_and_plot(dt, t_final, erk)
fig_phase.savefig("../figures/duffing__solution_rk4.pdf", bbox_inches="tight")
fig_legend.savefig("../figures/duffing__legend.pdf", bbox_inches="tight")

erk = bs5; dt = 0.25; t_final = 5000
tt, uu, tt_r, uu_r, fig_phase, fig_eta, fig_legend = compute_and_plot(dt, t_final, erk)
fig_phase.savefig("../figures/duffing__solution_bs5.pdf", bbox_inches="tight")